In [1]:
# !pip install chatterbot

  Using cached ChatterBot-1.0.5-py2.py3-none-any.whl (67 kB)
  Using cached python_dateutil-2.7.5-py2.py3-none-any.whl (225 kB)
  Using cached Pint-0.19.2.tar.gz (292 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached mathparse-0.1.2-py3-none-any.whl (7.2 kB)
  Using cached SQLAlchemy-1.2.19.tar.gz (5.7 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached PyYAML-5.1.2.tar.gz (265 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pymongo-3.12.3-cp39-cp39-win_amd64.whl (398 kB)
  Using cached spacy-2.1.9.tar.gz (30.7 MB)
  Installing build dependencies: start

  error: subprocess-exited-with-error
  
  pip subprocess to install build dependencies did not run successfully.
  exit code: 1
  
  [310 lines of output]
    Using cached setuptools-62.4.0-py3-none-any.whl (1.2 MB)
    Using cached wheel-0.32.3-py2.py3-none-any.whl (21 kB)
    Using cached Cython-0.29.30-py2.py3-none-any.whl (985 kB)
    Using cached cymem-2.0.6-cp39-cp39-win_amd64.whl (36 kB)
    Using cached preshed-2.0.1.tar.gz (113 kB)
    Preparing metadata (setup.py): started
    Preparing metadata (setup.py): finished with status 'done'
    Using cached murmurhash-1.0.7-cp39-cp39-win_amd64.whl (18 kB)
    Using cached thinc-7.0.8.tar.gz (1.9 MB)
    Preparing metadata (setup.py): started
    Preparing metadata (setup.py): finished with status 'done'
    Using cached blis-0.2.4.tar.gz (1.5 MB)
    Preparing metadata (setup.py): started
    Preparing metadata (setup.py): finished with status 'done'
    Using cached wasabi-0.9.1-py3-none-any.whl (26 kB)
    Using cached srsly-1.0

In [2]:
# !pip install streamlit

In [3]:
# !pip install streamlit-chat

In [7]:
import math
import numpy as np
import pandas as pd
import random
import re
import torch
import urllib.request
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast
from transformers import GPT2LMHeadModel

import streamlit as st
from streamlit_chat import message as st_message

from streamlit_chat import message
from chatterbot.trainers import ListTrainer

In [ ]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

model1 = torch.load('./data/GPT2_model/gpt_finance.pth')
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 

if 'past' not in st.session_state: # 내 입력채팅값 저장할 리스트
        st.session_state['past'] = [] 

if 'generated' not in st.session_state: # 챗봇채팅값 저장할 리스트
    st.session_state['generated'] = []

placeholder = st.empty() # 채팅 입력창을 아래위치로 내려주기위해 빈 부분을 하나 만듬

with st.form('form', clear_on_submit=True): # 채팅 입력창 생성
        user_input = st.text_input('당신: ', '') # 입력부분
        submitted = st.form_submit_button('전송') # 전송 버튼

if submitted and user_input:
    user_input1 = user_input.strip() # 채팅 입력값 및 여백제거
    if user_input1 == 'quit':
        st.stop()

    # output값
    a = ""
    while 1:
        input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + user_input1 + SENT + A_TKN + a)).unsqueeze(dim=0)
        pred = model1(input_ids)
        pred = pred.logits
        gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
        if gen == EOS:
            break
        a += gen.replace("▁", " ")

    chatbot_output1 = a.strip() # text generation된 값 및 여백 제거

    st.session_state.past.append(user_input1) # 입력값을 past 에 append -> 채팅 로그값 저장을 위해
    st.session_state.generated.append(chatbot_output1)
    
with placeholder.container(): # 리스트에 append된 채팅입력과 로봇출력을 리스트에서 꺼내서 메세지로 출력
    for i in range(len(st.session_state['past'])):
        st_message(st.session_state['past'][i], is_user=True, key=str(i) + '_user')
        if len(st.session_state['generated']) > i:
            st_message(st.session_state['generated'][i], key=str(i) + '_bot')


In [ ]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'



if 'past' not in st.session_state: # 내 입력채팅값 저장할 리스트
        st.session_state['past'] = [] 

if 'generated' not in st.session_state: # 챗봇채팅값 저장할 리스트
    st.session_state['generated'] = []

placeholder = st.empty() # 채팅 입력창을 아래위치로 내려주기위해 빈 부분을 하나 만듬

with st.form('form', clear_on_submit=True): # 채팅 입력창 생성
        user_input = st.text_input('당신: ', '') # 입력부분
        submitted = st.form_submit_button('전송') # 전송 버튼

if submitted and user_input:
    user_input1 = user_input.strip() # 채팅 입력값 및 여백제거
    if user_input1 == 'quit':
        st.stop()

    # output값
    a = ""
    while 1:
        input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + user_input1 + SENT + A_TKN + a)).unsqueeze(dim=0)
        pred = model1(input_ids)
        pred = pred.logits
        gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
        if gen == EOS:
            break
        a += gen.replace("▁", " ")

    chatbot_output1 = a.strip() # text generation된 값 및 여백 제거

    st.session_state.past.append(user_input1) # 입력값을 past 에 append -> 채팅 로그값 저장을 위해
    st.session_state.generated.append(chatbot_output1)
    
with placeholder.container(): # 리스트에 append된 채팅입력과 로봇출력을 리스트에서 꺼내서 메세지로 출력
    for i in range(len(st.session_state['past'])):
        st_message(st.session_state['past'][i], is_user=True, key=str(i) + '_user')
        if len(st.session_state['generated']) > i:
            st_message(st.session_state['generated'][i], key=str(i) + '_bot')


In [8]:
message("My message") 
message("Hello bot!", is_user=True)

2022-06-17 11:17:41.513 
  command:

    streamlit run C:\Users\bitcamp\Anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
